In [ ]:
import numpy as np
import os
import cv2
# 数据集在硬盘中的存储位置
dataset_path = "H:/Datasets/PetImages"
# 存储猫与狗的文件夹名称分别为Cat与Dog
categories = ['Cat', 'Dog']
# 图片经过处理后的尺寸
img_size = 100

In [ ]:
# 存储处理后的数据
dataset = []
for category in categories:
    # 依次获取到猫或者狗文件夹的位置
    folder_path = os.path.join(dataset_path, category)
    # 将猫的图片的标签设置为0，狗的图片的标签设置为1
    class_num = categories.index(category)
    # 获取到文件夹中全部的图片名字，并以列表的形式返回
    img_names = os.listdir(folder_path)
    # 依次读取文件夹中每一张图片，并对其进行处理
    for img_name in img_names:
        try:
            # 将文件夹路径与图片名称进行拼接获取到完整图片路径
            img_path = os.path.join(folder_path, img_name)
            # 读取图片
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            # 将图片的尺寸转换为100x100x3
            img = cv2.resize(img, (img_size, img_size))
            # 将处理好的图片与对应的标签存储在数据集中
            dataset.append((img, class_num))
        except Exception:
            pass

In [ ]:
import random
random.shuffle(dataset)

In [ ]:
# X存储处理好的图片，y存储图片对应的标签值
X = []
y = []
for img, label in dataset:
    X.append(img)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)
y = np.array(y)
np.save('X_cat_and_dog.npy', X)
np.save('y_cat_and_dog.npy', y)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X = np.load('X_cat_and_dog.npy')
# 将图片像素值进行归一化
X = X / 255.0
y = np.load('y_cat_and_dog.npy')
# 将数据集的75%划分为训练集，其余25%划分为测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
model = Sequential()
# 第1个卷积层与池化层
model.add(Conv2D(input_shape= X.shape[1:],
                 filters=32, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
# 第3个卷积层与池化层
model.add(Conv2D(filters=64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
# 第3个卷积层与池化层
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
# 第4个卷积层与池化层
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
# 扁平化层
model.add(Flatten())
# 全连接神经网络
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 在模型训练时应用早停法
from keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', 
                        patience=5,
                        mode='auto',
                        restore_best_weights=True)

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=['accuracy'])
model.fit(X_train,
          y_train,
          epochs=50,
          batch_size=64,
          verbose=2,
          callbacks=[monitor],
          validation_split=0.2)